In [ ]:
@file:DependsOn("/data/tools/actin-personalization/actin-personalization.jar")

import com.hartwig.actin.personalization.ncr.serialization.NcrDataReader

val records = NcrDataReader.read("/data/patient_like_me/ncr/latest/K2400223.csv")

In [ ]:
import com.hartwig.actin.personalization.datamodel.MetastasesDetectionStatus
import com.hartwig.actin.personalization.datamodel.Episode
import com.hartwig.actin.personalization.datamodel.ReferencePatient
import com.hartwig.actin.personalization.datamodel.Treatment
import com.hartwig.actin.personalization.ncr.interpretation.ReferencePatientFactory

fun Episode.doesNotIncludeAdjuvantOrNeoadjuvantTreatment(): Boolean {
    return !hasHadPreSurgerySystemicChemotherapy &&
            !hasHadPostSurgerySystemicChemotherapy &&
            !hasHadPreSurgerySystemicTargetedTherapy &&
            !hasHadPostSurgerySystemicTargetedTherapy
}

val patients = ReferencePatientFactory.default().create(records)

val referencePop = patients.flatMap(ReferencePatient::tumorEntries).map { (diagnosis, episodes) ->
    diagnosis to episodes.single { it.order == 1 }
}
    .filter { (_, episode) ->
        episode.distantMetastasesDetectionStatus == MetastasesDetectionStatus.AT_START &&
        // Comment out the next line (treatment filtering) to match R results:
        episode.systemicTreatmentPlan?.treatment?.let { it != Treatment.OTHER } == true &&
        episode.surgeries.isEmpty() &&
        episode.doesNotIncludeAdjuvantOrNeoadjuvantTreatment() &&
        episode.systemicTreatmentPlan?.observedPfsDays != null
    }
    .sortedBy { it.second.systemicTreatmentPlan!!.observedPfsDays!! }

val patientsByTreatment = referencePop.groupBy { (_, episode) ->
    episode.systemicTreatmentPlan!!.treatment.treatmentGroup
}
    .toList()
    .sortedByDescending { it.second.size }

In [ ]:
import com.hartwig.actin.personalization.similarity.population.DiagnosisAndEpisode

data class EventCountAndSurvivalAtTime(val daysSincePlanStart: Int, val numEvents: Int, val survival: Double)
    
tailrec fun eventAndCensorshipHistories(
    populationToProcess: List<DiagnosisAndEpisode>,
    eventHistory: List<EventCountAndSurvivalAtTime> = emptyList(),
    censorshipHistory: List<EventCountAndSurvivalAtTime> = emptyList()
): Pair<List<EventCountAndSurvivalAtTime>, List<EventCountAndSurvivalAtTime>> {
    return if (populationToProcess.isEmpty()) {
        eventHistory to censorshipHistory
    } else {
        val treatmentDetails = populationToProcess.first().second.systemicTreatmentPlan!!
        val previousEvent = eventHistory.lastOrNull() ?: EventCountAndSurvivalAtTime(0, 0, 1.0)
        val (newEventHistory, newCensorshipHistory) = if (treatmentDetails.hadProgressionEvent!!) {
            val newEvent = EventCountAndSurvivalAtTime(
                treatmentDetails.observedPfsDays!!, previousEvent.numEvents + 1, previousEvent.survival * (1 - (1.0 / populationToProcess.size))
            )
            Pair(eventHistory + newEvent, censorshipHistory)
        } else {
            val newCensorship = EventCountAndSurvivalAtTime(treatmentDetails.observedPfsDays!!, previousEvent.numEvents, previousEvent.survival)
            Pair(eventHistory, censorshipHistory + newCensorship)
        }
        eventAndCensorshipHistories(populationToProcess.drop(1), newEventHistory, newCensorshipHistory)
    }
}

val (eventHistory, censorshipHistory) = eventAndCensorshipHistories(referencePop)

In [ ]:
%use kandy

In [ ]:
// Compare with naive median days until progression
fun median(list: List<Int>): Double {
    return when(list.size) {
        0 -> Double.NaN
        1 -> list.first().toDouble()
        else -> {
            val midPoint = list.size / 2
            list.sorted().let {
                if (it.size % 2 == 0)
                    (it[midPoint] + it[midPoint - 1]) / 2.0
                else
                    it[midPoint].toDouble()
            }
        }
    }
}

val medianDaysUntilProgression = median(referencePop.mapNotNull { (_, episode) -> episode.systemicTreatmentPlan?.pfsDays })
println(medianDaysUntilProgression)
val (onlyProgressionHistory, _) = eventAndCensorshipHistories(referencePop.filter { (_, episode) ->
    episode.systemicTreatmentPlan?.pfsDays != null
})

val (censoredAsProgressionHistory, _) = eventAndCensorshipHistories(referencePop.map { (diagnosis, episode) ->
    diagnosis to episode.copy(systemicTreatmentPlan=episode.systemicTreatmentPlan?.copy(hadProgressionEvent=true))
})

val filteredHistory = eventHistory.filter { it.daysSincePlanStart <= 800 }
val filteredOnlyProgressionHistory = onlyProgressionHistory.filter { it.daysSincePlanStart <= 800 }
val filteredCensoredAsProgressionHistory = censoredAsProgressionHistory.filter { it.daysSincePlanStart <= 800 }
val combinedHistory = filteredHistory + filteredOnlyProgressionHistory + filteredCensoredAsProgressionHistory

val percentageArray = (0..10).toList().map { 0.1 * it to "${it * 10}%" }.toTypedArray()

plot {
    step {
        x(combinedHistory.map(EventCountAndSurvivalAtTime::daysSincePlanStart)) {
            axis {
                breaks((0..1000 step 50).toList())
                name = "Days since treatment start"
            }
        }
        y(combinedHistory.map(EventCountAndSurvivalAtTime::survival)) {
            axis {
                breaksLabeled(*percentageArray)
                name = "PFS %"
            }
        }
        color(filteredHistory.map { "Kaplan-Meier" } + filteredOnlyProgressionHistory.map { "Ignore censored" } + filteredCensoredAsProgressionHistory.map { "Censored as progression" }) {
            legend {
                name = "Method"
            }
        }
    }
    layout {
        size = 600 to 400
        theme {
            legend.position = LegendPosition.Top
        }
    }
}

In [ ]:
import com.hartwig.actin.personalization.datamodel.LocationGroup
import com.hartwig.actin.personalization.similarity.population.PopulationDefinition

val popDefinitions = PopulationDefinition.createAllForPatientProfile(50, 1, false, setOf(LocationGroup.PERITONEUM))
val popHistories = popDefinitions.map { (name, criteria) ->
    val (eventHistory, censorshipHistory) = eventAndCensorshipHistories(referencePop.filter(criteria))
    Triple(name, eventHistory.filter { it.daysSincePlanStart <= 800 }, censorshipHistory)
}

plot {
    step {
        x(popHistories.flatMap { (_, eventHistory, _) -> eventHistory.map(EventCountAndSurvivalAtTime::daysSincePlanStart) }) {
            axis {
                breaks((0..800 step 50).toList())
                name = "Days since treatment start"
            }
        }
        y(popHistories.flatMap { (_, eventHistory, _) -> eventHistory.map(EventCountAndSurvivalAtTime::survival) }, "PFS %") {
            axis.breaksLabeled(*percentageArray)
        }
        color(popHistories.flatMap { (name, eventHistory, _) -> eventHistory.map { name } }, "Population")
    }
    layout.size = 800 to 400
}

In [ ]:
popHistories.map { (name, eventHistory, _) -> name to eventHistory.firstOrNull { it.survival <= 0.5 } }
    .forEach { (name, eventCountAndSurvivalAtTime) -> println("$name: ${eventCountAndSurvivalAtTime?.daysSincePlanStart} days") }

In [ ]:
val historiesByTreatment = referencePop.groupBy { (_, episode) -> episode.systemicTreatmentPlan!!.treatment.treatmentGroup.display }
    .mapValues { (_, tumors) ->
        eventAndCensorshipHistories(tumors).first.filter { it.daysSincePlanStart <= 800 }
    }
    .filter { (_, histories) -> histories.size >= 20 }
plot {
    step {
        x(historiesByTreatment.flatMap { (_, histories) -> histories.map(EventCountAndSurvivalAtTime::daysSincePlanStart) }) {
            axis {
                breaks((0..800 step 50).toList())
                name = "Days since treatment start"
            }
        }
        y(historiesByTreatment.flatMap { (_, histories) -> histories.map(EventCountAndSurvivalAtTime::survival) }, "PFS %") {
            axis.breaksLabeled(*percentageArray)
        }
        color(historiesByTreatment.flatMap { (name, histories) -> histories.map { name } }, "Treatment")
    }
    layout.size = 800 to 400
}

In [ ]:
import org.jetbrains.kotlinx.kandy.ir.Plot

fun createPfsPlot(sortedPopulationsByName: Map<String, List<DiagnosisAndEpisode>>): Plot {
    val historiesByName = sortedPopulationsByName.mapValues { (_, tumors) ->
        eventAndCensorshipHistories(tumors).first
    }
    .filter { (_, histories) -> histories.size >= 20 }
    
    val longestInterval = historiesByName.values.map { it.last().daysSincePlanStart }.max()
    
    return plot {
        step {
            x(historiesByName.flatMap { (_, histories) -> histories.map(EventCountAndSurvivalAtTime::daysSincePlanStart) }) {
                axis {
                    breaks((0..longestInterval step 100).toList())
                    name = "Days since treatment start"
                }
            }
            y(historiesByName.flatMap { (_, histories) -> histories.map(EventCountAndSurvivalAtTime::survival) }, "PFS %") {
                axis.breaksLabeled(*percentageArray)
            }
            color(historiesByName.flatMap { (name, histories) -> histories.map { name } }, "Group")
        }
        layout.size = 800 to 400
    }
}

createPfsPlot(referencePop.groupBy { (_, episode) -> episode.systemicTreatmentPlan!!.treatment.treatmentGroup.display })

In [ ]:
historiesByTreatment.mapValues { (_, histories) -> histories.first { it.survival <= 0.75 } }.entries
    .sortedByDescending { (_, medianEvent) -> medianEvent.daysSincePlanStart!! }
    .forEach { (name, eventCountAndSurvivalAtTime) -> println("$name: ${eventCountAndSurvivalAtTime?.daysSincePlanStart} days") }

In [ ]:
historiesByTreatment.mapValues { (_, histories) ->
    val searchIndex = histories.binarySearchBy(-0.75) { -it.survival }
    histories[if (searchIndex < 0) -(searchIndex + 1) else searchIndex]
}.entries
    .sortedByDescending { (_, medianEvent) -> medianEvent.daysSincePlanStart!! }
    .forEach { (name, eventCountAndSurvivalAtTime) -> println("$name: ${eventCountAndSurvivalAtTime?.daysSincePlanStart} days") }

In [ ]:
%use dataframe

fun pfsForPopulation(population: List<DiagnosisAndEpisode>): String {
    val medianPfs = eventAndCensorshipHistories(population).first.firstOrNull { it.survival <= 0.5 }?.daysSincePlanStart
    return "$medianPfs (n=${population.size})"
}

fun dataFrame(rowLabels: List<Any?>, firstColumnName: String, namedColumns: List<Pair<String, List<String>>>): DataFrame<*> {
    val labelStrings = rowLabels.map { it?.let { it.toString() } ?: "None" }
    return (listOf(firstColumnName to labelStrings) + namedColumns).toMap().toDataFrame()
}

fun pfsTable(patientsByTreatment: Map<String, List<DiagnosisAndEpisode>>, columnDefinitions: List<PopulationDefinition>): DataFrame<*> {
    val allPatients = patientsByTreatment.flatMap { it.value }
    val sortedPatients = patientsByTreatment.entries.sortedByDescending { it.value.size }
                             
    val entries = columnDefinitions.map { (title, criteria) ->
        val populationSize: Int = allPatients.count(criteria)
        val annotatedTitle = "$title (n=$populationSize)"
        annotatedTitle to sortedPatients.map { pfsForPopulation(it.value.filter(criteria)) }
    }
    
    return dataFrame(sortedPatients.map { it.key }, "Treatment", entries)
}

val patientsByTreatment = referencePop.groupBy { (_, episode) -> episode.systemicTreatmentPlan!!.treatment.treatmentGroup.display }
pfsTable(patientsByTreatment, popDefinitions)